<a href="https://colab.research.google.com/github/hamednasr/tensorflow/blob/main/04_transfer_learning_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# transfer learning in tensorflow: feature extraction

## load data

In [1]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2022-02-27 18:41:13--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.181.128, 64.233.183.128, 173.194.194.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.181.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   195MB/s    in 0.8s    

2022-02-27 18:41:14 (195 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [2]:
import zipfile
zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

In [3]:
import os
for dirpath, dirnames,filenames in os.walk('10_food_classes_10_percent'):
  print(f'there are {len(dirnames)} folders and {len(filenames)} images in {dirpath}')

there are 2 folders and 0 images in 10_food_classes_10_percent
there are 10 folders and 0 images in 10_food_classes_10_percent/test
there are 0 folders and 250 images in 10_food_classes_10_percent/test/fried_rice
there are 0 folders and 250 images in 10_food_classes_10_percent/test/chicken_wings
there are 0 folders and 250 images in 10_food_classes_10_percent/test/pizza
there are 0 folders and 250 images in 10_food_classes_10_percent/test/steak
there are 0 folders and 250 images in 10_food_classes_10_percent/test/grilled_salmon
there are 0 folders and 250 images in 10_food_classes_10_percent/test/sushi
there are 0 folders and 250 images in 10_food_classes_10_percent/test/chicken_curry
there are 0 folders and 250 images in 10_food_classes_10_percent/test/ramen
there are 0 folders and 250 images in 10_food_classes_10_percent/test/hamburger
there are 0 folders and 250 images in 10_food_classes_10_percent/test/ice_cream
there are 10 folders and 0 images in 10_food_classes_10_percent/train


## preprocess the data

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_shape = (224, 224)
batch_size = 32

train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

train_datagen = ImageDataGenerator(rescale=1/255) 
test_datagen = ImageDataGenerator(rescale=1/255) 

print('Training images:')
train_data_10_percent = train_datagen.flow_from_directory(directory=train_dir,
                                                          target_size=image_shape,
                                                          batch_size=batch_size,
                                                          class_mode='categorical')
print('\nTest images:')
test_data = test_datagen.flow_from_directory(directory=test_dir,
                                              target_size=image_shape,
                                              batch_size=batch_size,
                                              class_mode='categorical')

Training images:
Found 750 images belonging to 10 classes.

Test images:
Found 2500 images belonging to 10 classes.


## create a tensorboard callback

In [5]:
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + '/' + experiment_name + '/' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f'saving tensorboard log files to: {log_dir}')
  return tensorboard_callback

## create models using tensorflow hub

In [6]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, optimizers

In [7]:
# two models from tensorflow hub

resnet_url = 'https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/5'
efficiennet_url='https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2'

In [8]:
def create_model(model_url, num_classes = 10):
  '''
  takes a tensorflow hub URL and creates a keras sequential model withit
  
  Args:
  model_url: the URL of model from tensorflow hub
  num_classes
  '''
  
  feature_extractor_layer = hub.KerasLayer(model_url,trainable=False,
                                           name='feature_extractor_layer',
                                           input_shape=image_shape+(3,))   
  
  model = tf.keras.Sequential([
                feature_extractor_layer,

                layers.Dense(num_classes, activation= 'softmax', name='output_layer')
      ])
  
  return model

### resnet model

In [9]:
resnet_model=create_model(resnet_url,train_data_10_percent.num_classes)

In [10]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extractor_layer (Ke  (None, 2048)             42626560  
 rasLayer)                                                       
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 42,647,050
Trainable params: 20,490
Non-trainable params: 42,626,560
_________________________________________________________________


In [11]:
resnet_model.compile(loss = 'categorical_crossentropy',
                     optimizer = optimizers.Adam(),
                     metrics = ['accuracy'])

In [12]:
resnet_model.fit(train_data_10_percent,epochs=20,
                 callbacks=[create_tensorboard_callback('tensorboard','resnet')])

saving tensorboard log files to: tensorboard/resnet/220227-184132
Epoch 1/20
24/24 [==============================] - 28s 356ms/step - loss: 1.8644 - accuracy: 0.3960
Epoch 2/20
24/24 [==============================] - 8s 311ms/step - loss: 0.8407 - accuracy: 0.7547
Epoch 3/20
24/24 [==============================] - 8s 312ms/step - loss: 0.5691 - accuracy: 0.8267
Epoch 4/20
24/24 [==============================] - 8s 314ms/step - loss: 0.4302 - accuracy: 0.8893
Epoch 5/20
24/24 [==============================] - 8s 313ms/step - loss: 0.3318 - accuracy: 0.9293
Epoch 6/20
24/24 [==============================] - 8s 339ms/step - loss: 0.2762 - accuracy: 0.9520
Epoch 7/20
24/24 [==============================] - 8s 316ms/step - loss: 0.2211 - accuracy: 0.9693
Epoch 8/20
24/24 [==============================] - 8s 320ms/step - loss: 0.1838 - accuracy: 0.9733
Epoch 9/20
24/24 [==============================] - 8s 318ms/step - loss: 0.1530 - accuracy: 0.9853
Epoch 10/20
24/24 [==============

In [13]:
resnet_model.evaluate(test_data)

79/79 [==============================] - 27s 329ms/step - loss: 0.5970 - accuracy: 0.8068


[0.596977710723877, 0.8068000078201294]

### efficient model

In [14]:
eff_model = create_model(efficiennet_url,train_data_10_percent.num_classes)

In [15]:
eff_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extractor_layer (Ke  (None, 1280)             5919312   
 rasLayer)                                                       
                                                                 
 output_layer (Dense)        (None, 10)                12810     
                                                                 
Total params: 5,932,122
Trainable params: 12,810
Non-trainable params: 5,919,312
_________________________________________________________________


In [16]:
eff_model.compile(loss = 'categorical_crossentropy',
                 optimizer=optimizers.Adam(),
                 metrics=['accuracy'])

In [17]:
eff_model.fit(train_data_10_percent,epochs=20,
                 callbacks=[create_tensorboard_callback('tensorboard','efficientnet')])

saving tensorboard log files to: tensorboard/efficientnet/220227-184525
Epoch 1/20
24/24 [==============================] - 14s 199ms/step - loss: 1.9866 - accuracy: 0.3413
Epoch 2/20
24/24 [==============================] - 4s 180ms/step - loss: 1.3125 - accuracy: 0.6840
Epoch 3/20
24/24 [==============================] - 4s 177ms/step - loss: 1.0071 - accuracy: 0.7440
Epoch 4/20
24/24 [==============================] - 4s 179ms/step - loss: 0.8426 - accuracy: 0.7920
Epoch 5/20
24/24 [==============================] - 4s 184ms/step - loss: 0.7348 - accuracy: 0.8200
Epoch 6/20
24/24 [==============================] - 4s 184ms/step - loss: 0.6526 - accuracy: 0.8480
Epoch 7/20
24/24 [==============================] - 5s 216ms/step - loss: 0.5888 - accuracy: 0.8680
Epoch 8/20
24/24 [==============================] - 5s 192ms/step - loss: 0.5385 - accuracy: 0.8853
Epoch 9/20
24/24 [==============================] - 5s 196ms/step - loss: 0.4944 - accuracy: 0.9067
Epoch 10/20
24/24 [========

In [18]:
eff_model.evaluate(test_data)

79/79 [==============================] - 17s 196ms/step - loss: 0.6107 - accuracy: 0.7944


[0.6107074022293091, 0.7943999767303467]

In [19]:
!nvidia-smi

Sun Feb 27 18:48:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    62W / 149W |   8457MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
!tensorboard dev upload --logdir ./tensorboard/ \
--name "efffient net vs. resnet" \
--description 'comparing two different tensorflow hub feature extraction model architechture' \
--one_shot




New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/YQvKfa5ZS66UqBWy8nMHgQ/

[2022-02-27T18:53:57] Started scanning logdir.
[2022-02-27T18:53:59] Total uploaded: 80 scalars, 0 tensors, 2 binary objects (5.8 MB)
[2022-02-27T18:53:59] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/YQvKfa5ZS66UqBWy8nMHgQ/


In [22]:
!tensorboard dev list

https://tensorboard.dev/experiment/YQvKfa5ZS66UqBWy8nMHgQ/
	Name                 efffient net vs. resnet
	Description          comparing two different tensorflow hub feature extraction model architechture
	Id                   YQvKfa5ZS66UqBWy8nMHgQ
	Created              2022-02-27 18:53:57 (12 minutes ago)
	Updated              2022-02-27 18:53:59 (12 minutes ago)
	Runs                 2
	Tags                 3
	Scalars              80
	Tensor bytes         0
	Binary object bytes  6050481
Total: 1 experiment(s)
